In [5]:
!pip install langgraph langchain_openai reportlab langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 14.9 MB/s eta 0:00:00


In [16]:
import os, re
from typing import Dict, TypedDict, Optional, List, Any, Annotated
from langgraph.graph import START, StateGraph, END
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import ToolNode, tools_condition, create_react_agent
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from google.colab import userdata
from langchain_groq import ChatGroq
from groq import Groq
from typing import TypedDict, List
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.units import inch
from google.colab import files


In [17]:
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

llm = "llama-3.1-8b-instant"
chat_groq_llm = ChatGroq(model_name=llm, groq_api_key=GROQ_API_KEY)
if not GROQ_API_KEY:
  raise ValueError("GROQ_API_KEY key not found in colab secrets. Please add it or give access.")

In [18]:
class ReportState(TypedDict):
    rewritten_output: str
    formatted_text: Optional[str]
    pdf_path: Optional[str]
    final_response: Optional[str]

In [19]:
def formatted_pdf(content:str, filename: str) -> str:
  file_path = f"/content/{filename}"

  doc = SimpleDocTemplate(file_path, pagesize=letter,
                          leftMargin= 1*inch, bottomMargin= 1*inch,
                          topMargin = 1*inch, rightMargin = 1*inch)

  styles = getSampleStyleSheet()
  title_style = ParagraphStyle(
      'CustomTitle',
      parent=styles['Heading1'],
      spaceAfter = 30,
      alignment = 1
  )
  body_style = styles['BodyText']

  story = []

  title = Paragraph("Research Report", title_style)
  story.append(title)

  lines = content.split('\n')
  for line in lines:
    line = line.strip()
    if not line:
      continue
    story.append(Paragraph(line, body_style))

  doc.build(story)
  return file_path

In [22]:
def report_workflow():
  graph = StateGraph(ReportState)
  graph.add_node("report_generator", generator_node)
  graph.set_entry_point("report_generator")
  graph.add_edge("report_generator",END)
  return graph.compile()

In [24]:
def download_pdf(pdf_path:str):
  try:
    files.download_pdf(pdf_path)
    print("PDF download started")
  except Exception as e:
    print(f"error downloading PDF: {e} ")

In [25]:
def generator_node(state: ReportState) -> ReportState:
  rewritten_output = state["rewritten_output"]
  prompt = f"""
  You are a report formatter. You take responses that include
  - A structured research plan, potentially including:
    - A breakdown of the topic into sub-questions.
    - Suggested keywords for searching
    - Key points from the original prompt to keep in mind.
  Format the following information into a structured, clear, and professional report PDF.
  Content:
  {rewritten_output}
  """

  response = chat_groq_llm.invoke(prompt)
  final_response = response.content.strip()

  pdf_path = formatted_pdf(final_response,"Research_report.pdf")

  print(f"PDF report Generated {pdf_path}")

  return {
      "rewritten_output": state["rewritten_output"],
      "formatted_text":state.get("formatted_text"),
      "pdf_path": pdf_path,
      "final_response": final_response

  }

In [26]:
if __name__ == "__main__":
    # ✅ Example input
    sample_input = {
        "rewritten_output": """
        Title: Global Renewable Energy Outlook 2025

        Renewable energy adoption has surged globally.
        Solar power capacity grew by 28% year-over-year, while wind energy now contributes 19%
        of total electricity generation worldwide.

        Challenges persist in grid storage, balancing, and supply chain bottlenecks.

        Future opportunities:
        - AI-driven energy optimization
        - Offshore wind expansion
        - Recycling lithium-ion batteries for sustainability
        """
    }

    workflow = report_workflow()
    result = workflow.invoke(sample_input)

    print("\n=== FINAL REPORT ===")
    print(result["final_response"])
    print(f"\nPDF saved at: {result['pdf_path']}")

    try:
        files.download(result["pdf_path"])
        print("✅ PDF download initiated.")
    except Exception as e:
        print(f"⚠️ Could not download PDF automatically: {e}")


PDF report Generated /content/Research_report.pdf

=== FINAL REPORT ===
**Global Renewable Energy Outlook 2025**

**Executive Summary**

The global renewable energy landscape is experiencing a significant surge, with solar power capacity growing by 28% year-over-year and wind energy contributing 19% of total electricity generation worldwide. Despite these advancements, challenges persist in grid storage, balancing, and supply chain bottlenecks. This report provides an outlook on the future of renewable energy, highlighting opportunities and areas for further development.

**Research Plan**

### Topic Breakdown

To better understand the current state and future prospects of renewable energy, the research topic will be broken down into the following sub-questions:

1. **What are the current trends and growth rates of solar and wind energy globally?**
2. **What are the key challenges facing the renewable energy sector, particularly in grid storage, balancing, and supply chain bottlenecks?

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ PDF download initiated.
